<a href="https://colab.research.google.com/github/kuds/reinforce-tactics/blob/main/notebooks/bot_tournament.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏆 Bot Tournament - Reinforce Tactics

Run automated tournaments between SimpleBot, MediumBot, and AdvancedBot on multiple maps!

**Features:**
- 🤖 Tournament between SimpleBot, MediumBot, and AdvancedBot
- 🗺️ Multiple map support with configurable map pool modes
- 📊 Comprehensive statistics: wins, losses, draws, win rates, Elo ratings
- 📈 Per-map performance analysis
- 💾 Downloadable results (JSON, CSV, replay files)

**Tournament Configuration:**
- **Bots**: SimpleBot, MediumBot, AdvancedBot (automatically discovered)
- **Maps**: 20x20_starter, 10x12_funnel_point, 18x18_center_mountains
- **Map Pool Mode**: `all` - plays every map for each matchup
- **Games Per Side**: 2 (4 games per map per matchup)
- **Total Games**: 36 games (3 matchups × 3 maps × 4 games)

**Quick Start:**
1. Run the setup cell to install dependencies and clone the repository
2. Run the tournament cell
3. View results and download artifacts


## 🔧 Setup: Install Dependencies and Clone Repository

In [ ]:
import os
import sys

# Check if we're in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 Running in Google Colab - Setting up environment...\n")
    
    # Clone the repository if not already present
    if not os.path.exists('reinforce-tactics'):
        print("📦 Cloning reinforce-tactics repository...")
        !git clone https://github.com/kuds/reinforce-tactics.git
        print("✅ Repository cloned successfully\n")
    else:
        print("✅ Repository already cloned\n")
    
    # Change to repository directory
    os.chdir('reinforce-tactics')
    
    # Install dependencies
    print("📦 Installing dependencies from requirements.txt...")
    !pip install -q -r requirements.txt
    print("✅ Dependencies installed successfully\n")
    
    # Add repository to Python path
    repo_path = os.getcwd()
    if repo_path not in sys.path:
        sys.path.insert(0, repo_path)
    
    print(f"✅ Setup complete! Working directory: {os.getcwd()}")
else:
    print("ℹ️  Not running in Colab - assuming local environment")
    print(f"Working directory: {os.getcwd()}")

## 🏆 Run Tournament

This cell runs a comprehensive tournament between SimpleBot, MediumBot, and AdvancedBot.

**Tournament Details:**
- 3 bot matchups (SimpleBot vs MediumBot, SimpleBot vs AdvancedBot, MediumBot vs AdvancedBot)
- 3 maps per matchup (20x20_starter.csv, 10x12_funnel_point.csv, 18x18_center_mountains.csv)
- 4 games per map (2 with each bot as Player 1)
- **Total: 36 games**

The tournament typically takes 2-5 minutes to complete.

In [ ]:
import subprocess
import sys

# Run the tournament using the tournament script
print("🏆 Starting Bot Tournament...\n")
print("Tournament Configuration:")
print("  - Bots: SimpleBot, MediumBot, AdvancedBot")
print("  - Maps: 20x20_starter, 10x12_funnel_point, 18x18_center_mountains")
print("  - Map Pool Mode: all (plays every map for each matchup)")
print("  - Games Per Side: 2 (4 games per map)")
print("  - Total Games: 36\n")
print("=" * 80)
print()

# Run tournament script
result = subprocess.run([
    sys.executable,
    'scripts/tournament.py',
    '--maps',
    'maps/1v1/20x20_starter.csv',
    'maps/1v1/10x12_funnel_point.csv',
    'maps/1v1/18x18_center_mountains.csv',
    '--map-pool-mode', 'all',
    '--games-per-side', '2',
    '--output-dir', 'tournament_results'
], capture_output=False, text=True)

if result.returncode == 0:
    print("\n✅ Tournament completed successfully!")
else:
    print(f"\n❌ Tournament failed with exit code: {result.returncode}")

## 📊 View Tournament Results

In [ ]:
import json
import pandas as pd
from IPython.display import display, HTML

# Read and display CSV results
print("📊 Tournament Results Summary:\n")
csv_path = 'tournament_results/tournament_results.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    display(df)
else:
    print("❌ Results file not found. Please run the tournament first.")

# Read and display detailed JSON results
json_path = 'tournament_results/tournament_results.json'
if os.path.exists(json_path):
    with open(json_path, 'r') as f:
        results = json.load(f)
    
    print("\n" + "=" * 80)
    print("📈 Per-Map Performance:")
    print("=" * 80)
    
    if 'per_map_summary' in results:
        for map_name, summary in results['per_map_summary'].items():
            print(f"\n🗺️  {map_name}")
            print(f"   Best Performer: {summary['best_performer']}")
            print(f"   Avg Game Length: {summary['avg_game_length']:.1f} turns")
    
    print("\n" + "=" * 80)
    print("📊 Elo Rating History:")
    print("=" * 80)
    
    if 'elo_history' in results:
        for bot_name, history in results['elo_history'].items():
            print(f"\n{bot_name}:")
            print(f"   Starting: {history[0]:.0f}")
            print(f"   Final: {history[-1]:.0f}")
            print(f"   Change: {history[-1] - history[0]:+.0f}")
else:
    print("\n❌ JSON results file not found.")

## 💾 Download Results and Replays

Download the tournament results and replay files to analyze offline.

In [ ]:
import zipfile
from pathlib import Path

# Create a zip file with all results
zip_path = 'tournament_results.zip'
results_dir = Path('tournament_results')

if results_dir.exists():
    print("📦 Creating zip archive of tournament results...")
    
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Add JSON and CSV results
        for file in ['tournament_results.json', 'tournament_results.csv']:
            file_path = results_dir / file
            if file_path.exists():
                zipf.write(file_path, f'tournament_results/{file}')
        
        # Add all replay files
        replays_dir = results_dir / 'replays'
        if replays_dir.exists():
            for replay_file in replays_dir.glob('*.json'):
                zipf.write(replay_file, f'tournament_results/replays/{replay_file.name}')
    
    print(f"✅ Archive created: {zip_path}")
    print(f"   Size: {os.path.getsize(zip_path) / 1024:.1f} KB")
    
    # In Colab, provide download link
    if IN_COLAB:
        from google.colab import files
        print("\n⬇️  Downloading archive...")
        files.download(zip_path)
        print("✅ Download complete!")
    else:
        print(f"\n💾 Archive saved to: {os.path.abspath(zip_path)}")
else:
    print("❌ Tournament results directory not found. Please run the tournament first.")

## ℹ️ Additional Information

**Bot Descriptions:**
- **SimpleBot**: Basic rule-based bot with straightforward tactics
- **MediumBot**: Improved bot with better strategic planning
- **AdvancedBot**: Sophisticated bot with advanced combat coordination

**Map Descriptions:**
- **20x20_starter.csv**: Standard balanced map for competitive play
- **10x12_funnel_point.csv**: Tactical map with strategic chokepoints
- **18x18_center_mountains.csv**: Map with central mountain terrain affecting movement

**Understanding Results:**
- **Win Rate**: Percentage of games won
- **Elo Rating**: Chess-like rating system (higher is better, starts at 1500)
- **Elo Change**: Change in rating during the tournament
- **Per-Map Stats**: Performance breakdown by individual map

**Replay Files:**
The replay files can be used to visualize games using the game's replay viewer.
